In [1]:
# https://www.youtube.com/watch?v=i3uMhH2xeOM&ab_channel=Buynomics

In [2]:
# https://ngugijoan.medium.com/pricing-on-point-the-art-and-science-of-dynamic-pricing-dd543bf80f01
# https://ngugijoan.medium.com/dynamic-pricing-implementation-through-data-science-price-optimization-strategies-56adab4d3176
# https://levelup.gitconnected.com/calculating-individual-price-elasticity-for-products-9787e3b82875
# https://www.kaggle.com/code/arnabchaki/flight-fare-prediction-0-96-r2-score?fbclid=IwZXh0bgNhZW0CMTAAAR05L4by3xyhImYsDOnF-ufsQQ7VbBefv8Bg3ECHy1JHCR_XmjSZIAKM7yE_aem_AWdIsN4qMSlU9R0FQsAR9y8hT_e_ggs_tIfnGdUdpwA4mLwPAbLPidigOsMMcKNF-4wyLjSg2hcmqzefdb3gX5bT
# https://datascience.oneoffcoder.com/pricing-elasticity-modeling.html#Random-forest

In [3]:
import warnings
warnings.simplefilter("ignore")

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

In [5]:
from sklearn.tree import DecisionTreeRegressor

In [6]:
from sklearn.model_selection import GridSearchCV

In [7]:
from sklearn.metrics import mean_squared_error

# 10. Load data

In [9]:
df = pd.read_csv("02. Labeled_retail_data.csv")

In [10]:
df.shape

(407629, 11)

# 11. Exploratory data analysis (EDA)

Count of Transactions by Description

In [13]:
category_counts = df['Category'].value_counts()
category_counts = category_counts.sort_values(ascending=False)

In [14]:
category_counts

Category
Home and Lifestyle      219974
Arts and Leisure         92162
Fashion and Travel       52586
Education and Office     24345
Health and Wellness      17759
Technology and More        803
Name: count, dtype: int64

Count of Transactions by Description

In [16]:
description_counts = df['Description'].value_counts()
description_counts = description_counts.sort_values(ascending=False)

In [17]:
description_counts

Description
WHITE HANGING HEART T-LIGHT HOLDER     3153
JUMBO BAG RED WHITE SPOTTY             1742
REGENCY CAKESTAND 3 TIER               1705
PACK OF 72 RETRO SPOT CAKE CASES       1586
STRAWBERRY CERAMIC TRINKET BOX         1407
                                       ... 
CHUNKY CRACKED GLAZE NECKLACE IVORY       1
GOLD CHRISTMAS STOCKING DECORATION        1
WHITE CHRISTMAS TREE 60CM                 1
S/16 BLACK SHINY/MAT BAUBLES              1
BAKING MOULD EASTER EGG MILK CHOC         1
Name: count, Length: 3979, dtype: int64

Count of Transactions by Invoice

In [19]:
invoice_counts = df['Invoice'].value_counts()
invoice_counts = invoice_counts.sort_values(ascending=False)

In [20]:
invoice_counts

Invoice
500356    270
511522    255
531382    251
507235    250
511051    248
         ... 
520316      1
520823      1
536568      1
534469      1
522060      1
Name: count, Length: 19178, dtype: int64

# 12. Feature engineering

In [22]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,profit_margin,Cost_price,Category
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,0.193525,5.605004,Home and Lifestyle
1,489559,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 12:55:00,6.95,17056.0,United Kingdom,0.400739,4.164866,Home and Lifestyle
2,489576,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,5,2009-12-01 13:38:00,7.95,15984.0,United Kingdom,0.062370,7.454161,Home and Lifestyle
3,489582,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,2,2009-12-01 13:47:00,7.95,14543.0,United Kingdom,0.448154,4.387179,Home and Lifestyle
4,489656,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 17:28:00,6.95,17428.0,United Kingdom,0.458394,3.764161,Home and Lifestyle


In [23]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [24]:
df['yyyymmdd'] = df['InvoiceDate'].dt.strftime('%Y%m%d')

datetime features

In [26]:
def datetime_feature_extraction(df):
    # Day extraction
    df['dayofweek'] = df['InvoiceDate'].dt.dayofweek
    df['dayofmonth'] = df['InvoiceDate'].dt.day
    df['dayofyear'] = df['InvoiceDate'].dt.dayofyear
    df['is_weekend'] = (df['InvoiceDate'].dt.dayofweek >= 5).astype(int)
    df['weekday_weekend'] = df['InvoiceDate'].dt.dayofweek.apply(lambda x: 0 if x >= 5 else 1)

    # Week extraction
    df['weekofyear'] = df['InvoiceDate'].dt.isocalendar().week
    df['weekofmonth'] = np.ceil(df['dayofmonth'] / 7).astype(int)    
    
    # Month extraction
    df['month'] = df['InvoiceDate'].dt.month
    df['quarter'] = df['InvoiceDate'].dt.quarter
    df['days_in_month'] = df['InvoiceDate'].dt.days_in_month

    # Year extraction
    df['year'] = df['InvoiceDate'].dt.year
    return df

In [27]:
df = datetime_feature_extraction(df)

group by

In [29]:
aggregation_functions = {
    'Quantity': 'sum',
    'Price': 'mean',
    'Cost_price':'mean',
    'dayofweek': 'max',
    'dayofmonth': 'max',
    'dayofyear': 'max',
    'is_weekend': 'max',
    'weekday_weekend': 'max',
    'weekofyear': 'max',
    'weekofmonth': 'max',
    'month': 'max',
    'quarter': 'max',
    'days_in_month': 'max',
    'year': 'max'
}

In [30]:
df = df.groupby(['StockCode','Category','Description','yyyymmdd']).agg(aggregation_functions).reset_index()

Create change

In [32]:
df['change_qty'] = df['Quantity'].pct_change()

In [33]:
df['change_price'] = df['Price'].pct_change()

In [34]:
df.dropna(inplace=True)

In [35]:
df.sample(10)

,StockCode,Category,Description,yyyymmdd,Quantity,Price,Cost_price,dayofweek,dayofmonth,dayofyear,is_weekend,weekday_weekend,weekofyear,weekofmonth,month,quarter,days_in_month,year,change_qty,change_price
103181,22175,Arts and Leisure,PINK OWL SOFT TOY,20100221,1,2.950000,2.702246,6,21,52,1,0,7,3,2,1,28,2010,0.000000,0.000000
94480,22085,Arts and Leisure,PAPER CHAIN KIT SKULLS,20100922,2,2.950000,2.149924,2,22,265,0,1,38,4,9,3,30,2010,-0.833333,0.000000
130124,22453,Arts and Leisure,MEASURING TAPE RUSSIAN DOLL BLUE,20100630,1,2.950000,1.692392,2,30,181,0,1,26,5,6,2,30,2010,-0.500000,0.000000
127172,22423,Home and Lifestyle,REGENCY CAKESTAND 3 TIER,20100820,26,12.492857,8.656893,4,20,232,0,1,33,3,8,3,31,2010,-0.653333,0.015679
109300,22236,Home and Lifestyle,CAKE STAND 3 TIER MAGIC GARDEN,20100415,2,12.750000,10.710382,3,15,105,0,1,15,3,4,2,30,2010,0.000000,0.000000
185512,82600,Home and Lifestyle,NO SINGING METAL SIGN,20101209,15,2.100000,1.625311,3,9,343,0,1,49,2,12,4,31,2010,-0.825581,0.069610
78422,21899,Fashion and Travel,KEY FOBGARAGE DESIGN,20100607,2,0.650000,0.422916,0,7,158,0,1,23,1,6,2,30,2010,-0.900000,0.000000
21163,20985,Technology and More,HEART CALCULATOR,20101027,3,1.250000,0.703861,2,27,300,0,1,43,4,10,4,31,2010,2.000000,0.000000
189508,84378,Home and Lifestyle,SET OF 3 HEART COOKIE CUTTERS,20101004,12,1.250000,0.754408,0,4,277,0,1,40,1,10,4,31,2010,-0.755102,0.000000
186193,84029C,Home and Lifestyle,BROWN FURRY HOT WATER BOTTLE,20091223,2,3.750000,2.790819,2,23,357,0,1,52,4,12,4,31,2009,-0.500000,0.000000


# 13. Store current price and quantity

In [37]:
df.sort_values(by=['Category','Description', 'yyyymmdd'], ascending=[True,True, False], inplace=True)

In [38]:
df['row_number'] = df.groupby('Description').cumcount() + 1

In [39]:
df.head()

,StockCode,Category,Description,yyyymmdd,Quantity,Price,Cost_price,dayofweek,dayofmonth,dayofyear,...,weekday_weekend,weekofyear,weekofmonth,month,quarter,days_in_month,year,change_qty,change_price,row_number
164251,35962,Arts and Leisure,12 ASS ZINC CHRISTMAS DECORATIONS,20101020,11,2.1,1.747029,2,20,293,...,1,42,3,10,4,31,2010,1.200000,0.0,1
164250,35962,Arts and Leisure,12 ASS ZINC CHRISTMAS DECORATIONS,20101010,5,2.1,1.878906,6,10,283,...,0,40,2,10,4,31,2010,1.500000,0.0,2
164249,35962,Arts and Leisure,12 ASS ZINC CHRISTMAS DECORATIONS,20101007,2,2.1,1.229558,3,7,280,...,1,40,1,10,4,31,2010,-0.833333,0.0,3
164248,35962,Arts and Leisure,12 ASS ZINC CHRISTMAS DECORATIONS,20101006,12,2.1,1.068459,2,6,279,...,1,40,1,10,4,31,2010,0.000000,0.0,4
164247,35962,Arts and Leisure,12 ASS ZINC CHRISTMAS DECORATIONS,20101005,12,2.1,1.289836,1,5,278,...,1,40,1,10,4,31,2010,1.000000,0.0,5


lastest

In [150]:
df_lastest = df[df['row_number'] == 1].reset_index(drop=True)

In [152]:
df_lastest.shape

(3979, 21)

In [154]:
df_lastest.head()

,StockCode,Category,Description,yyyymmdd,Quantity,Price,Cost_price,dayofweek,dayofmonth,dayofyear,...,weekday_weekend,weekofyear,weekofmonth,month,quarter,days_in_month,year,change_qty,change_price,row_number
0,35962,Arts and Leisure,12 ASS ZINC CHRISTMAS DECORATIONS,20101020,11,2.10,1.747029,2,20,293,...,1,42,3,10,4,31,2010,1.200000,0.0,1
1,22436,Arts and Leisure,12 COLOURED PARTY BALLOONS,20101209,20,0.65,0.329037,3,9,343,...,1,49,2,12,4,31,2010,0.666667,0.0,1
2,21440,Arts and Leisure,12 MINI TOADSTOOL PEGS,20100701,13,1.25,0.974084,3,1,182,...,1,26,1,7,3,31,2010,1.600000,0.0,1
3,84465,Arts and Leisure,15 PINK FLUFFY CHICKS IN BOX,20100406,1,2.95,1.575270,1,6,96,...,1,14,1,4,2,30,2010,0.000000,0.0,1
4,21458,Arts and Leisure,2 PICTURE BOOK EGGS EASTER BUNNY,20100827,12,1.25,0.749582,4,27,239,...,1,34,4,8,3,31,2010,11.000000,0.0,1


# 6. Model

split df by category

In [46]:
df['Category'].unique()

array(['Arts and Leisure', 'Education and Office', 'Fashion and Travel',
       'Health and Wellness', 'Home and Lifestyle', 'Technology and More'],
      dtype=object)

In [47]:
df_art_leisure = df[df['Category'] == 'Arts and Leisure']

In [48]:
df_education_office = df[df['Category'] == 'Education and Office']

In [49]:
df_fashion_travel = df[df['Category'] == 'Fashion and Travel']

In [50]:
df_health_wellness = df[df['Category'] == 'Health and Wellness']

In [51]:
df_home_lifestyle = df[df['Category'] == 'Home and Lifestyle']

In [52]:
df_tech = df[df['Category'] == 'Technology and More']

StockCode	Category	Description	yyyymmdd	Quantity	Price	Cost_pricesplit X and y

In [54]:
X_art_leisure = df_art_leisure.drop(columns=['StockCode','Category','Description','yyyymmdd','Quantity','Price','Cost_price','row_number','change_qty'])
y_art_leisure = df_art_leisure['change_qty']

In [55]:
print(X_art_leisure.shape, y_art_leisure.shape)

(51303, 12) (51303,)


In [56]:
X_education_office = df_education_office.drop(columns=['StockCode','Category','Description','yyyymmdd','Quantity','Price','Cost_price','row_number','change_qty'])
y_education_office = df_education_office['change_qty']

In [57]:
print(X_education_office.shape, y_education_office.shape)

(15537, 12) (15537,)


In [58]:
X_fashion_travel = df_fashion_travel.drop(columns=['StockCode','Category','Description','yyyymmdd','Quantity','Price','Cost_price','row_number','change_qty'])
y_fashion_travel = df_fashion_travel['change_qty']

In [59]:
print(X_fashion_travel.shape, y_fashion_travel.shape)

(27787, 12) (27787,)


In [60]:
X_health_wellness = df_health_wellness.drop(columns=['StockCode','Category','Description','yyyymmdd','Quantity','Price','Cost_price','row_number','change_qty'])
y_health_wellness = df_health_wellness['change_qty']

In [61]:
print(X_health_wellness.shape, y_health_wellness.shape)

(9023, 12) (9023,)


In [62]:
X_home_lifestyle = df_home_lifestyle.drop(columns=['StockCode','Category','Description','yyyymmdd','Quantity','Price','Cost_price','row_number','change_qty'])
y_home_lifestyle = df_home_lifestyle['change_qty']

In [63]:
print(X_home_lifestyle.shape, y_home_lifestyle.shape)

(119297, 12) (119297,)


In [64]:
X_tech = df_tech.drop(columns=['StockCode','Category','Description','yyyymmdd','Quantity','Price','Cost_price','row_number','change_qty'])
y_tech = df_tech['change_qty']

In [65]:
print(X_tech.shape, y_tech.shape)

(625, 12) (625,)


model training (art and leisure)

In [67]:
rt = DecisionTreeRegressor(random_state=42)

In [68]:
param_grid = {
    'max_depth': [5, 10],
    'min_samples_split': [10, 15],
    'min_samples_leaf': [5, 10]
}

In [69]:
grid_search = GridSearchCV(estimator=rt, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

In [70]:
grid_search.fit(X_art_leisure, y_art_leisure)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


GridSearchCV(cv=3, estimator=DecisionTreeRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 10], 'min_samples_leaf': [5, 10],
                         'min_samples_split': [10, 15]},
             scoring='neg_mean_squared_error', verbose=2)

In [71]:
grid_search.best_score_

-2528.433195113989

In [72]:
best_model_art_leisure = grid_search.best_estimator_

model training (education and office)

In [74]:
rt = DecisionTreeRegressor(random_state=42)

In [75]:
param_grid = {
    'max_depth': [5, 10],
    'min_samples_split': [10, 15],
    'min_samples_leaf': [5, 10]
}

In [76]:
grid_search = GridSearchCV(estimator=rt, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

In [77]:
grid_search.fit(X_education_office, y_education_office)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


GridSearchCV(cv=3, estimator=DecisionTreeRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 10], 'min_samples_leaf': [5, 10],
                         'min_samples_split': [10, 15]},
             scoring='neg_mean_squared_error', verbose=2)

In [78]:
grid_search.best_score_

-5746.836645797142

In [79]:
best_model_education_office = grid_search.best_estimator_

model training (fashion and travel)

In [81]:
rt = DecisionTreeRegressor(random_state=42)

In [82]:
param_grid = {
    'max_depth': [5, 10],
    'min_samples_split': [10, 15],
    'min_samples_leaf': [5, 10]
}

In [83]:
grid_search = GridSearchCV(estimator=rt, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

In [84]:
grid_search.fit(X_fashion_travel, y_fashion_travel)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


GridSearchCV(cv=3, estimator=DecisionTreeRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 10], 'min_samples_leaf': [5, 10],
                         'min_samples_split': [10, 15]},
             scoring='neg_mean_squared_error', verbose=2)

In [85]:
grid_search.best_score_

-281.52899579404874

In [86]:
best_model_fashion_travel = grid_search.best_estimator_

model training (health and wellness)

In [88]:
rt = DecisionTreeRegressor(random_state=42)

In [89]:
param_grid = {
    'max_depth': [5, 10],
    'min_samples_split': [10, 15],
    'min_samples_leaf': [5, 10]
}

In [90]:
grid_search = GridSearchCV(estimator=rt, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

In [91]:
grid_search.fit(X_health_wellness, y_health_wellness)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


GridSearchCV(cv=3, estimator=DecisionTreeRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 10], 'min_samples_leaf': [5, 10],
                         'min_samples_split': [10, 15]},
             scoring='neg_mean_squared_error', verbose=2)

In [92]:
grid_search.best_score_

-2555.2471719008267

In [93]:
best_model_health_wellness = grid_search.best_estimator_

model training (home and lifestyle)

In [95]:
rt = DecisionTreeRegressor(random_state=42)

In [96]:
param_grid = {
    'max_depth': [5, 10],
    'min_samples_split': [10, 15],
    'min_samples_leaf': [5, 10]
}

In [97]:
grid_search = GridSearchCV(estimator=rt, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

In [98]:
grid_search.fit(X_home_lifestyle, y_home_lifestyle)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


GridSearchCV(cv=3, estimator=DecisionTreeRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 10], 'min_samples_leaf': [5, 10],
                         'min_samples_split': [10, 15]},
             scoring='neg_mean_squared_error', verbose=2)

In [99]:
grid_search.best_score_

-726.0673858563101

In [100]:
best_model_home_lifestyle = grid_search.best_estimator_

model training (tech)

In [102]:
rt = DecisionTreeRegressor(random_state=42)

In [103]:
param_grid = {
    'max_depth': [5, 10],
    'min_samples_split': [10, 15],
    'min_samples_leaf': [5, 10]
}

In [104]:
grid_search = GridSearchCV(estimator=rt, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

In [105]:
grid_search.fit(X_tech, y_tech)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


GridSearchCV(cv=3, estimator=DecisionTreeRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 10], 'min_samples_leaf': [5, 10],
                         'min_samples_split': [10, 15]},
             scoring='neg_mean_squared_error', verbose=2)

In [106]:
grid_search.best_score_

-74.98847997707055

In [107]:
best_model_tech = grid_search.best_estimator_

In [108]:
**

SyntaxError: invalid syntax (402238255.py, line 1)

# 7. Optimize

optimize (art and leisure)

In [215]:
for index, row in df_lastest.iterrows():
    print(index)
    df_optimize = df_lastest[:index+1][['dayofweek','dayofmonth','dayofyear','is_weekend',
                                        'weekday_weekend','weekofyear','weekofmonth','month',
                                        'quarter','days_in_month','year']]
    multipliers = [round(x * 0.1, 1) for x in range(-9, 11)]
    df_optimize = df_optimize.loc[np.repeat(df_optimize.index.values, len(multipliers))]
    df_optimize['change_price'] = multipliers

    change_price_list = []
    for i in range(df_optimize.shape[0]):
        if row['Category'] == 'Arts and Leisure':
            demand_change = best_model_art_leisure.predict(df_optimize.iloc[[i]])
        elif row['Category'] == 'Education and Office':
            demand_change = best_model_education_office.predict(df_optimize.iloc[[i]])
        elif row['Category'] == 'Fashion and Travel':
            demand_change = best_model_fashion_travel.predict(df_optimize.iloc[[i]])
        elif row['Category'] == 'Health and Wellness':
            demand_change = best_model_health_wellness.predict(df_optimize.iloc[[i]])
        elif row['Category'] == 'Home and Lifestyle':
            demand_change = best_model_home_lifestyle.predict(df_optimize.iloc[[i]])
        elif row['Category'] == 'Technology and More':
            demand_change = best_model_tech.predict(df_optimize.iloc[[i]])        

        change_price_list.append(demand_change[0])
        
    df_optimize['change_qty'] = change_price_list

    initial_price = df_lastest[df_lastest['StockCode'] == row['StockCode']]['Price'].iloc[0]
    initial_cost_price = df_lastest[df_lastest['StockCode'] == row['StockCode']]['Cost_price'].iloc[0]
    initial_quantity = df_lastest[df_lastest['StockCode'] == row['StockCode']]['Quantity'].iloc[0]

    df_optimize['price_new'] = initial_price*(1+df_optimize['change_price'])
    df_optimize['qty_new'] = initial_quantity*(1+df_optimize['change_qty'])
    df_optimize['additional_profit'] = df_optimize['qty_new']*(df_optimize['price_new']-initial_cost_price)

    df_optimize = df_optimize[df_optimize['additional_profit'] == df_optimize['additional_profit'].max()]


    print(df_optimize)
        
    break

0
   dayofweek  dayofmonth  dayofyear  is_weekend  weekday_weekend  weekofyear  \
0          2          20        293           0                1          42   

   weekofmonth  month  quarter  days_in_month  year  change_price  change_qty  \
0            3     10        4             31  2010          -0.1   17.617462   

   price_new    qty_new  additional_profit  
0       1.89  204.79208          29.279287  


In [ ]:
**

In [ ]:
df_sku_optimize = df_sku[['dayofweek','dayofmonth','dayofyear','is_weekend','weekday_weekend','weekofyear','weekofmonth','month','quarter','days_in_month','year']].sample(1)

In [185]:
multipliers = [round(x * 0.1, 1) for x in range(1, 11)]

In [ ]:
df_sku_optimize = df_sku_optimize.loc[np.repeat(df_sku_optimize.index.values, len(multipliers))]

In [ ]:
df_sku_optimize['change_price'] = multipliers

In [ ]:
df_sku_optimize.head()

optimize

In [ ]:
change_price_list = []

In [ ]:
for i in range(df_sku_optimize.shape[0]):
    demand_change = best_model.predict(df_sku_optimize.iloc[[i]])
    change_price_list.append(demand_change[0])

In [ ]:
df_sku_optimize['change_qty'] = change_price_list

In [ ]:
initial_price = df_price[df_price['Description'] == 'REGENCY CAKESTAND 3 TIER']['Price'].iloc[0]
initial_cost_price = df_price[df_price['Description'] == 'REGENCY CAKESTAND 3 TIER']['Cost_price'].iloc[0]
initial_quantity = df_price[df_price['Description'] == 'REGENCY CAKESTAND 3 TIER']['Quantity'].iloc[0]

In [ ]:
initial_price

In [ ]:
df_sku_optimize['price_new'] = initial_price*(1+df_sku_optimize['change_price'])

In [ ]:
df_sku_optimize['qty_new'] = initial_quantity*(1+df_sku_optimize['change_qty'])

In [ ]:
df_sku_optimize['additional_profit'] = df_sku_optimize['qty_new']*(df_sku_optimize['price_new']-initial_cost_price)

In [ ]:
df_sku_optimize.head()

select max additional profit

In [ ]:
df_sku_optimize[df_sku_optimize['additional_profit'] == df_sku_optimize['additional_profit'].max()]